# Gas Storage Valuation 

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

BASE_DIR = Path('..').resolve()
DATA_RAW = BASE_DIR / 'data' / 'raw'
RESULTS = BASE_DIR / 'results'
RESULTS.mkdir(parents=True, exist_ok=True)
print('BASE_DIR:', BASE_DIR)

BASE_DIR: C:\Users\dappy\Downloads\Public-portfolio\Energy\gas_storage_valuation


In [2]:
import sys


# notebook add-on, to find repo root by walking up until we see src
p = Path().resolve()
while p != p.parent and not (p / "src").exists():
    p = p.parent

sys.path.insert(0, str(p))
print("Added to path:", p)

Added to path: C:\Users\dappy\Downloads\Public-portfolio\Energy\gas_storage_valuation


## Extrinsic-style value (Monte Carlo - upper bound)
apply the optimiser to each simulated path. Because each path is optimised with full future knowledge,
this produces an  upper bound on value. 

In [3]:
from src.price_models import fit_ou_params, build_seasonal_theta, simulate_ou_paths
from src.storage_dp import StorageParams
from src.valuation_utils import monte_carlo_upper_bound

spot = pd.read_csv(DATA_RAW / 'spot_history_ttf_like.csv', parse_dates=['date']).sort_values('date')
logp = np.log(spot['spot_eur_mwh']).dropna()
ou = fit_ou_params(logp)
dates = pd.DatetimeIndex(spot["date"])
theta = build_seasonal_theta(dates)


start = pd.to_datetime(spot["date"].iloc[-1]) + pd.Timedelta(days=1)
future_dates = pd.date_range(start, periods=365, freq="D")
theta = build_seasonal_theta(future_dates)

paths = simulate_ou_paths(
    s0=float(spot["spot_eur_mwh"].iloc[-1]),
    params=ou,
    n_days=365,
    n_paths=200,
    seed=7,
    seasonal_theta=theta
)

params = StorageParams(
    capacity=100.0,
    init_inventory=50.0,
    inj_rate=2.0,
    wdr_rate=2.0,
    inj_fee=0.02,
    wdr_fee=0.02,
    loss_frac=0.0,
    discount_rate_annual=0.0,   # or 0.03 for discountng
)

# future_dates must have length == n_days (same as simulated paths rows)
res = monte_carlo_upper_bound(
    paths=paths,
    dates=future_dates,
    params=params,
    max_paths=200
)
res.head()


,path,npv_eur,npv_eur_mean,npv_eur_p50,npv_eur_p10,npv_eur_p90
0,0,2601.128341,5527.615657,4698.210143,2869.153656,9211.402856
1,1,9174.541140,5527.615657,4698.210143,2869.153656,9211.402856
2,2,2776.409253,5527.615657,4698.210143,2869.153656,9211.402856
3,3,3549.273839,5527.615657,4698.210143,2869.153656,9211.402856
4,4,4090.183756,5527.615657,4698.210143,2869.153656,9211.402856


In [4]:
summary = res['npv_eur'].describe(percentiles=[0.05, 0.5, 0.95]).to_frame('npv_eur')
summary


,npv_eur
count,200.000000
mean,5527.615657
std,2642.091402
min,1589.056787
5%,2508.278720
50%,4698.210143
95%,10101.481284
max,17892.903914


In [5]:
out = RESULTS / 'storage_value_mc_upper_bound_notebook.csv'
res.to_csv(out, index=False)
print('Wrote', out)

Wrote C:\Users\dappy\Downloads\Public-portfolio\Energy\gas_storage_valuation\results\storage_value_mc_upper_bound_notebook.csv


### Sensitivity: vary injection cost

In [6]:
sens = []
for inj_cost in [0.00, 0.01, 0.02, 0.05]:
    p = params
    p = StorageParams(**{**p.__dict__, 'inj_fee': inj_cost})
    tmp = monte_carlo_upper_bound(paths, future_dates, p, max_paths=50)    # fewer paths for speed
    sens.append({'inj_cost': inj_cost, 'mean_npv': tmp['npv_eur'].mean(), 'p05': tmp['npv_eur'].quantile(0.05), 'p95': tmp['npv_eur'].quantile(0.95)})
sens_df = pd.DataFrame(sens)
sens_df

,inj_cost,mean_npv,p05,p95
0,0.00,4541.693463,2296.212790,8262.949107
1,0.01,4538.518897,2291.228315,8260.065031
2,0.02,4535.363291,2288.242531,8257.186843
3,0.05,4526.570776,2279.683022,8249.580206
